In [1]:
import numpy as np
import itertools

### What to do
2 by 2
[[],[],
[],[]]
(black card, p1 cards, p2 cards)

In [2]:
def dominationcheck(X, m):
    nrows = X.shape[0]
    ncols = X.shape[1]

    output = []
    if m == 'cols':
        for i in range(0, ncols-1):
            for j in range(i+1, ncols):
                index1 = (X[:, i] < X[:, j]).sum()
                index2 = (X[:, i] > X[:, j]).sum()
                
                if index1 == nrows:
                    output.append(i)
                if index2 == nrows:
                    output.append(j)

    elif m == 'rows':
        for i in range(0, nrows-1):
            for j in range(i+1, nrows):
                index1 = (X[i, :] < X[j, :]).sum()
                index2 = (X[i, :] > X[j, :]).sum()

                if index1 == ncols:
                    output.append(i)
                if index2 == ncols:
                    output.append(j)

    return output

In [3]:
def dominationdelete(A, B, moves=False):
    rows = dominationcheck(A, 'rows')
    cols = dominationcheck(B, 'cols')
    A = np.delete(A, rows, axis=0)
    A = np.delete(A, cols, axis=1)
    B = np.delete(B, rows, axis=0)
    B = np.delete(B, cols, axis=1)

    if moves:
        return A, B, rows, cols
    else:
        return A, B

In [4]:
def dominationfinal(A, B, moves=False):
    removedrows = []
    removedcols = []
    while (A.shape != dominationdelete(A, B, moves)[0].shape) and (B.shape != dominationdelete(A, B, moves)[1].shape) :
        result = dominationdelete(A, B, moves)
        A = result[0]
        B = result[1]

        if moves:
            removedrows += result[2]
            removedcols += result[3]

    return A, B, removedrows, removedcols

In [5]:
def solve2by2(payout_a, payout_b, moves=False):
    if (payout_a[0] == payout_a[1]).all() and (payout_b[:, 0] == payout_b[:, 1]).all():
        result = (payout_a[0, 0], payout_b[0, 0])
        strategy = ([1,1],[1,1])
        return result, strategy
    
    A, B = np.ones((2,2)), np.ones((2,2))
    A[1,0], A[1,1] = payout_b[0,0]-payout_b[0,1], payout_b[1,0]-payout_b[1,1]
    B[1,0], B[1,1] = payout_a[0,0]-payout_a[1,0], payout_a[0,1]-payout_a[1,1]
    

    result = dominationfinal(payout_a, payout_b, moves)

    if result[0].shape != payout_a.shape:

        if moves:
            p_a, p_b = np.ones(2), np.ones(2)
            for i in result[2]:
                p_a[i] = 0
            for j in result[3]:
                p_b[j] = 0

            strategy = (list(p_a), list(p_b))
            return (result[0].sum() / max(result[0].shape), result[1].sum() / max(result[1].shape)), strategy
        else:
            return (result[0].sum() / max(result[0].shape), result[1].sum() / max(result[1].shape))

    else:
        p_a = np.linalg.solve(A, np.array([1, 0]))
        p_b = np.linalg.solve(B, np.array([1, 0]))
    
        result = (p_b[0] * payout_a[0, 0] + p_b[1] * payout_a[0, 1], p_a[0] * payout_b[0, 0] + p_a[1] * payout_b[1, 0])

        if moves:
            strategy = (list(p_a), list(p_b))
            return result, strategy
        else:
            return result


In [12]:
def payout2by2(c, a, b, moves=False):
    c1, c2 = c[0], c[1]
    
    payout_a = np.zeros((2,2))
    payout_b = np.zeros((2,2))
    
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x > y:
                payout_a[i,j] += c1
                payout_a[(i+1)%2,(j+1)%2] += c2
            elif x == y:
                payout_a[i,j] += c1/2
                payout_b[i,j] += c1/2
                payout_a[(i+1)%2,(j+1)%2] += c2/2
                payout_b[(i+1)%2, (j+1)%2] += c2/2
            else:
                payout_b[i,j] += c1
                payout_b[(i+1)%2,(j+1)%2] += c2
    
    #print(payout_a)
    #print(payout_b)
    
    return solve2by2(payout_a, payout_b, moves)

In [7]:
def solve2byN(payout_a, payout_b, moves=False):
    c = payout_a[0, 0] + payout_b[0, 0]
    cols = np.array([z for z in range(payout_a.shape[1])])
    strategy_a = {}
    payoffs_b = {}

    combs = list(itertools.combinations(cols, 2))

    for i in combs:
        nash = np.array(i)
        check = np.setdiff1d(cols, nash)

        result = solve2by2(payout_a[:, nash], payout_b[:, nash], moves=True)
        
        for j in check:
            if sum(result[1][0]) > 1:
                test = min(payout_b[:, j])
                    
            else:
                test = np.matmul(result[1][0], payout_b[:, j])
            

            if result[0][1] >= test:
                strategy = np.zeros(payout_a.shape[1])
                ind = 0
                for k in nash:
                    strategy[k] = result[1][1][ind]
                    ind += 1

                strategy_a[tuple(strategy)] = result[1][0]
                payoffs_b[tuple(strategy)] = result[0][1]


    max_payoff_b = max(payoffs_b.values())
    max_strategy = []

    for key, val in payoffs_b.items():
        if val == max_payoff_b:
            max_strategy.append(key)

    if len(max_strategy) > 1:
        index = [sum(k) for k in max_strategy]
        l = 0
        for ind in index:
            if ind == max(index):
                max_strategy = max_strategy[l]
                break
            l += 1
    else:
        max_strategy = max_strategy[0]

    return (c-max_payoff_b, max_payoff_b), (strategy_a[max_strategy], list(max_strategy))
        
        




In [8]:
pa = np.array([[2,0,4,0],[0,1,1,0]])
pb = np.array([[1,0,-1,0],[0,2,3,0]])

solve2byN(pa, pb)

((2.0, 1.0), ([1.0, 0.0], [1.0, 0.0, 0.0, 0.0]))

In [17]:
def payout3by3(c, a, b, moves=False):
    # c = [cards that appear in order]
    # a = [a cards]
    # b = [b cards]
    c_original = c.copy()
    a_original = a.copy()
    b_original = b.copy()
    c1, c2, c3 = c[0], c[1], c[2]
    # initialize payout matrix
    payout_a = np.zeros((3,3))
    payout_b = np.zeros((3,3))
    # fill payout matrix
    for i, x in enumerate(a_original):
        for j, y in enumerate(b_original):
            if x > y:
                #print(c, c1, x, y)
                c.remove(c1)
                a.remove(x)
                b.remove(y)
                
                payout_a[i,j] += payout2by2(c, a, b)[0] + c1
                payout_b[i,j] += payout2by2(c, a ,b)[1]
            if x == y:
                #print(c, c1, x, y)
                c.remove(c1)
                a.remove(x)
                b.remove(y)

                payout_a[i,j] += payout2by2(c, a, b)[0] + (c1/2)
                payout_b[i,j] += payout2by2(c, a, b)[1] + (c1/2)
            if x < y:
                #print(c, c1, x, y)
                c.remove(c1)
                a.remove(x)
                b.remove(y)
                
                payout_a[i,j] += payout2by2(c, a, b)[0] 
                payout_b[i,j] += payout2by2(c, a, b)[1] + c1
            c, a, b = c_original.copy(), a_original.copy(), b_original.copy()
    return payout_a, payout_b
            

In [10]:
def solve3by3(payout_a, payout_b, moves=False):
    # case when game is singular and irreducible
    if payout_a[0] == payout_a[1] == payout_a[2] and payout_b[:,0] == payout_b[:,1] == payout_b[:,2]:
        result = (payout_a[0,0], payout_b[0,0])
        strategy = ([1,1,1], [1,1,1])
        if moves:
            return result, strategy
        else: 
            return result
    
    p_a, p_b = np.ones(1,3), np.ones(1,3)
    result = dominationfinal(payout_a, payout_b, moves)
    
    if result[0].shape == (1,1):
        # take the singular equilibrium
        out_a = result[0][0]
        out_b = result[1][0]
        if moves:
            p_a, p_b = np.ones(3)
            for i in result[2]:
                p_a[i] = 0
            for j in result[3]:
                p_b[j] = 0
            return (out_a, out_b), (p_a, p_b)
        ret = (out_a, out_b)

    elif result[0].shape == (1,2) or result[0].shape == (2,1):
        # take highest value
        ret = (result[0].sum() / max(result[0].shape), result[1].sum() / max(result[1].shape))
        if moves:
            p_a, p_b = np.ones(3)
            for i in result[2]:
                p_a[i] = 0
            for j in result[3]:
                p_b[j] = 0
            return ret, (p_a, p_b)
        return ret

    elif result[0].shape == (1,3) or result[0].shape == (1,3):
        # take highest value
        ret = (result[0].sum() / max(result[0].shape), result[1].sum() / max(result[1].shape))
        if moves:
            p_a, p_b = np.ones(3)
            for i in result[2]:
                p_a[i] = 0
            for j in result[3]:
                p_b[j] = 0
            return ret, (p_a, p_b)
        return ret

    elif result[0].shape == (2,2):
        # use solve2by2
        return solve2by2(result[0], result[1], moves)
    
    elif result[0].shape == (2,3):
        # use solve2byN
        return solve2byN(result[0], result[1], moves)

    elif result[0].shape == (3,2):
        # use solve 2byN
        b, a, p_b, p_a = solve2byN(result[1].T, result[0].T, moves)[0][0],
                         solve2byN(result[1].T, result[0].T, moves)[0][1], 
                         solve2byN(result[1].T, result[0].T, moves)[1][0], 
                         solve2byN(result[1].T, result[0].T, moves)[1][1]
        if moves:
             return (a, b), (p_a, p_b)
        return (a, b)
    
    else:
        # solve the system of equations
        A, B = np.ones((3,3)), np.ones((3,3))
        A[1,0], A[1,1], A[1,2] = payout_b[0,0]-payout_b[0,1], payout_b[1,0]-payout_b[1,1], payout_b[2,0]-payout_b[2,1]
        A[1,0], A[1,1], A[1,2] = payout_b[0,0]-payout_b[0,2], payout_b[1,0]-payout_b[1,2], payout_b[2,0]-payout_b[2,2]

        B[1,0], B[1,1], B[1,2] = payout_a[0,0]-payout_a[1,0], payout_a[0,1]-payout_a[1,1], payout_a[0,2]-payout_a[1,2] 
        B[2,0], B[2,1], B[2,2] = payout_a[0,0]-payout_a[2,0], payout_a[0,1]-payout_a[2,1], payout_a[0,2]-payout_a[2,2]

        p_a = np.linalg.solve(A, np.array([1, 0, 0]))
        p_b = np.linalg.solve(B, np.array([1, 0, 0])) 
        
        ret = (p_b[0]*payout_a[0, 0] + p_b[1]*payout_a[0, 1] + p_b[2]*payout-a[0, 2], p_a[0]*payout_b[0, 0] + p_a[1]*payout_b[1, 0] + p_a[2]*payout_b[2, 0])

        if moves:
             return ret, (p_a, p_b)
        return ret



IndentationError: expected an indented block (2863590139.py, line 54)

In [4]:
a = [(1,2), (3,4)]
b = *a

SyntaxError: can't use starred expression here (<ipython-input-4-5b3236aa20b2>, line 2)

In [18]:
a, b = payout3by3([1,2,3], [1,2,3], [1,2,3])
a, b

(array([[3. , 3.2, 4.4],
        [2.8, 3. , 3.2],
        [1.6, 2.8, 3. ]]),
 array([[3. , 2.8, 1.6],
        [3.2, 3. , 2.8],
        [4.4, 3.2, 3. ]]))